In [1]:
import pymongo
import os
import pandas as pd


In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covidcrypto_db

In [3]:
bitcoin_file = os.path.join("Resources", "Bitcoin", "rev_bitcoin2020data.csv")
bitcoin_df = pd.read_csv(bitcoin_file)
bitcoin_df.head()

,Unnamed: 0,Timestamp,Block,Volume_(BTC),Volume_(Currency),Weighted_Price
0,0,2020-01-01,4202896,0.669749,"$4,807.37","$7,177.87"
1,1,2020-01-02,4204336,0.510742,"$3,637.32","$7,121.64"
2,2,2020-01-03,4205776,12.028600,"$88,234.41","$7,335.39"
3,3,2020-01-04,4207216,5.121394,"$37,530.58","$7,328.20"
4,4,2020-01-05,4208656,0.020840,$154.70,"$7,423.24"


In [4]:
daywise_file = os.path.join("Resources", "COVID-19","day_wise.csv")
daywise_df = pd.read_csv(daywise_file)
daywise_df = daywise_df.rename(columns={"Date": "Timestamp"})




In [5]:
merge_df = daywise_df.merge(bitcoin_df, how='outer', on='Timestamp')

In [17]:
db.data.drop()

In [18]:
for index, row in merge_df.iterrows():
    db.data.insert_one(
        {
            "date": row["Timestamp"],
            "bitcoin_price": row["Weighted_Price"],
            "world_new_cases": row["New cases"],
            "world_new_deaths": row["New deaths"],
            "world_new_recovered": row["New recovered"],
            "world_confirmed": row["Confirmed"],
            "world_deaths": row["Deaths"],
            "world_recovered": row["Recovered"],
            "world_active": row["Active"],       
            "countries": []
        }
    )

In [19]:
csv_file = os.path.join("Resources", "COVID-19", "full_grouped.csv")
full_df = pd.read_csv(csv_file)
full_df.head()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa


In [20]:
for index, row in full_df.iterrows():
    db.data.update_one({'date': str(row["Date"])},
                       {'$push': {"countries": {
                           "country_name": row["Country/Region"],
                           "country_confirmed": row["Confirmed"],
                           "country_deaths": row["Deaths"],
                           "country_recovered": row["Recovered"],
                           "country_active": row["Active"],
                           "country_new_cases": row["New cases"],
                           "country_new_deaths": row["New deaths"],
                           "country_new_recovered": row["New recovered"],
                               }
                           }
                       })